# Local development and docker image components

- This section assumes that you have already created a program to perform the task required in a particular step of your ML workflow. We will again use the MINIST toy model for demonstration.

- Then, a Docker container image will be built to package your program. 

- Then, call `kfp.components.ContainerOp` to convert it to a component that can be used in a pipeline. Your component can create outputs that the downstream components can use as inputs

**Note: Make sure that you have docker installed in the local environment**

In [1]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp
import datetime

import kubernetes as k8s

In [6]:
PROJECT_ID='kubeflow-pipeline-fantasy'

## Wrap an existing Docker container image using `ContainerOp`
Since **docker** is not installed in the kubeflow notebook, the following cells cannot be run.

### Create a Docker container
Create your own container image that includes your program. 
- If your component creates some outputs to be fed as inputs to the downstream components, each separate output must be written as a string to a separate local text file inside the container image. 
- For example, if a trainer component needs to output the trained model path, it can write the path to a local file `/output.txt`. 
- The string written to an output file cannot be too big. If it is too big (>> 100 kB), it is recommended to save the output to an external persistent storage and pass the storage path to the next component.

The following cell creates a file `app.py` that contains a Python script. The script takes a GCS bucket name as an input argument, gets the lists of blobs in that bucket, prints the list of blobs and also writes them to an output file.

In [30]:
%%bash

# Create folders if they don't exist.
mkdir -p tmp/components/minist_training

# Create the Python file that lists GCS blobs.
cat > ./tmp/components/minist_training/app.py <<HERE
import argparse
from datetime import datetime
import tensorflow as tf

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model_file', type=str, required=True, help='Name of the model file.')
parser.add_argument(
    '--bucket', type=str, required=True, help='GCS bucket name.')
args = parser.parse_args()

bucket=args.bucket
model_file=args.model_file

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())    

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir=bucket + '/logs/' + datetime.now().date().__str__()),
  # Interrupt training if val_loss stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
]

model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(x_test, y_test))


model.save(model_file)

from tensorflow import gfile

gcs_path = bucket + "/" + model_file

if gfile.Exists(gcs_path):
    gfile.Remove(gcs_path)

gfile.Copy(model_file, gcs_path)
with open('/output.txt', 'w') as f:
  f.write(gcs_path)
HERE

Now create a container that runs the script. Start by creating a `Dockerfile`. A `Dockerfile` contains the instructions to assemble a Docker image. The `FROM` statement specifies the Base Image from which you are building. `WORKDIR` sets the working directory. When you assemble the Docker image, `COPY` will copy the required files and directories (for example, `app.py`) to the filesystem of the container. `RUN` will execute a command (for example, install the dependencies) and commits the results. 

In [31]:
%%bash

# Create Dockerfile.
cat > ./tmp/components/minist_training/Dockerfile <<EOF
FROM tensorflow/tensorflow:1.15.0-py3
WORKDIR /app
COPY . /app
EOF

Now that we have created our Dockerfile we can create our Docker image. Then we need to push the image to a registry to host the image. Now create a Shell script that builds a container image and stores it in the Google Container Registry.

In [32]:
%%bash -s "{PROJECT_ID}"

IMAGE_NAME="minist_training_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"

# Create script to build docker image and push it.
cat > ./tmp/components/minist_training/build_image.sh <<HERE
PROJECT_ID="${1}"
IMAGE_NAME="${IMAGE_NAME}"
TAG="${TAG}"
GCR_IMAGE="gcr.io/\${PROJECT_ID}/\${IMAGE_NAME}:\${TAG}"
docker build -t \${IMAGE_NAME} .
docker tag \${IMAGE_NAME} \${GCR_IMAGE}
docker push \${GCR_IMAGE}
docker image rm \${IMAGE_NAME}
docker image rm \${GCR_IMAGE}
HERE

Run the script.

In [33]:
%%bash

# Build and push the image.
cd tmp/components/minist_training
bash build_image.sh


Step 1/3 : FROM tensorflow/tensorflow:1.15.0-py3
 ---> f24a5ca8605f
Step 2/3 : WORKDIR /app
 ---> Running in 360f66865c09
Removing intermediate container 360f66865c09
 ---> a8a9b4e8797e
Step 3/3 : COPY . /app
 ---> 14fb2833cc41
Successfully built 14fb2833cc41
Successfully tagged minist_training_kf_pipeline:latest
The push refers to repository [gcr.io/kubeflow-pipeline-fantasy/minist_training_kf_pipeline]
7d4309fa235b: Preparing
8d2418a6ceee: Preparing
84c3bc63b701: Preparing
56ec85ad394c: Preparing
aefe991487a2: Preparing
4a58ecdd995f: Preparing
fa9f3f4bd775: Preparing
2bf9e296738e: Preparing
92486bede3ce: Preparing
19331eff40f0: Preparing
100ef12ce3a4: Preparing
97e6b67a30f1: Preparing
a090697502b8: Preparing
19331eff40f0: Waiting
100ef12ce3a4: Waiting
4a58ecdd995f: Waiting
97e6b67a30f1: Waiting
fa9f3f4bd775: Waiting
a090697502b8: Waiting
2bf9e296738e: Waiting
92486bede3ce: Waiting
56ec85ad394c: Layer already exists
aefe991487a2: Layer already exists
84c3bc63b701: Layer already exist

### Define each component
Define a component by creating an instance of `kfp.dsl.ContainerOp` that describes the interactions with the Docker container image created in the previous step. You need to specify the component name, the image to use, the command to run after the container starts, the input arguments, and the file outputs. .

In [34]:
def minist_train_op(model_file, bucket):
    return dsl.ContainerOp(
      name="minist_training_container",
      image='gcr.io/{}/minist_training_kf_pipeline:latest'.format(PROJECT_ID),
      command=['python', '/app/app.py'],
      file_outputs={'outputs': '/output.txt'},
      arguments=['--bucket', bucket, '--model_file', model_file]
    )

### Create your workflow as a Python function

Define your pipeline as a Python function. ` @kfp.dsl.pipeline` is a required decoration including `name` and `description` properties. Then compile the pipeline function. After the compilation is completed, a pipeline file is created.

In [35]:
# Define the pipeline
@dsl.pipeline(
   name='Minist pipeline',
   description='A toy pipeline that performs minist model training.'
)
def minist_container_pipeline(
    model_file: str = 'mnist_model.h5', 
    bucket: str = "gs://kubeflow-pipeline-fantasy-kubeflow1-bucket"
):
    minist_train_op(model_file=model_file, bucket=bucket).apply(gcp.use_gcp_secret('user-gcp-sa'))

In [36]:
#Get or create an experiment and submit a pipeline run
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    host = "https://kubeflow1.endpoints.kubeflow-pipeline-fantasy.cloud.goog/pipeline"
    client_id = "493831447550-os23o55235htd9v45a9lsejv8d1plhd0.apps.googleusercontent.com"
    other_client_id = "493831447550-iu24vv6id3ng5smhf2lboovv5qukuhbh.apps.googleusercontent.com"
    other_client_secret = "cB8Xj-rb9JWCYcCRDlpTMfhc"
    client = kfp.Client(host=host, 
                        client_id=client_id,
                        other_client_id=other_client_id, 
                        other_client_secret=other_client_secret)

In [37]:
pipeline_func = minist_container_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

compiler.Compiler().compile(pipeline_func, pipeline_filename)
#Submit a pipeline run
arguments = {"model_file":"mnist_model.h5",
             "bucket":"gs://kubeflow-pipeline-fantasy-kubeflow1-bucket"}
run_name = pipeline_func.__name__ + ' run'
experiment = client.create_experiment('python-functions-minist')

run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

Follow the [instructions](https://www.kubeflow.org/docs/other-guides/accessing-uis/) on kubeflow.org to access Kubeflow UIs. Upload the created pipeline and run it.

**Warning:** When the pipeline is run, it pulls the image from the repository to the Kubernetes cluster to create a container. Kubernetes caches pulled images. One solution is to use the image digest instead of the tag in your component dsl, for example, `s/v1/sha256:9509182e27dcba6d6903fccf444dc6188709cc094a018d5dd4211573597485c9/g`. Alternatively, if you don't want to update the digest every time, you can try `:latest` tag, which will force the k8s to always pull the latest image..

___